# Pregunta 1

Dado que yo uso el entorno de Visual Studio Code:

In [7]:
!pip install selenium

# Llamado a librería webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import requests
import re
from lxml import html
import pandas as pd


# Pregunta 2 y 3

In [8]:
#Pregunta 2 y 3
browser = webdriver.Chrome()
browser.get('https://books.toscrape.com/catalogue/category/books/science-fiction_16/index.html')

# Pregunta 4:

Primero, extraigo el Título:


![imagen1](https://github.com/rodrigosilupu/DMC_WS/blob/main/p4_titulo.png?raw=true)



In [9]:
#título
title=browser.find_element(By.XPATH, '//ol/li[1]/article/h3/a').get_attribute('title')
print(title)

Mesaerion: The Best Science Fiction Stories 1800-1849


Luego, extraigo el Precio:

![imagen2](https://github.com/rodrigosilupu/DMC_WS/blob/main/p4_precio.png?raw=true)



In [10]:
#precio
price=browser.find_element(By.XPATH, '//ol/li[1]/article/div[@class="product_price"]/p[1]').text
price= re.search(r'(\d+\.\d+)', price).group(1)
print(price)

37.59


Luego extraigo la disponibilidad:

![imagen3](https://github.com/rodrigosilupu/DMC_WS/blob/main/p4_disponibilidad.png?raw=true)

In [11]:
#disponibilidad
availability=browser.find_element(By.XPATH, '//ol/li[1]/article/div[@class="product_price"]/p[2]').text
print(availability)

In stock


El número de estrellas:

![Imagen4](https://github.com/rodrigosilupu/DMC_WS/blob/main/p4_rating.png?raw=true)

In [12]:
# Nro de estrellas:
rating_class = browser.find_element(By.XPATH, '//ol/li[1]/article/p').get_attribute('class').split()[1]
def text_to_number(text):
    mapping = {
        "Zero": 0,
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }
    return mapping.get(text, 0)
number_of_stars = text_to_number(rating_class)
print(number_of_stars)

1


El link del libro:

![img10](https://github.com/rodrigosilupu/DMC_WS/blob/main/p4_link.png?raw=true)

In [13]:
#link del libro
link=browser.find_element(By.XPATH, '//ol/li[1]/article/h3/a').get_attribute('href')
print(link)

https://books.toscrape.com/catalogue/mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html


Finalmente, la descripción del libro:

![img20](https://github.com/rodrigosilupu/DMC_WS/blob/main/p4_descipcion.png?raw=true)

In [14]:
#description
rl=requests.get(link)
tree_detail=html.fromstring(rl.content)
description=tree_detail.xpath('//article/p')[0].text
print(description)

Andrew Barger, award-winning author and engineer, has extensively researched forgotten journals and magazines of the early 19th century to locate groundbreaking science fiction short stories in the English language. In doing so, he found what is possibly the first science fiction story by a female (and it is not from Mary Shelley). Andrew located the first steampunk short Andrew Barger, award-winning author and engineer, has extensively researched forgotten journals and magazines of the early 19th century to locate groundbreaking science fiction short stories in the English language. In doing so, he found what is possibly the first science fiction story by a female (and it is not from Mary Shelley). Andrew located the first steampunk short story, which has not been republished since 1844. There is the first voyage to the moon in a balloon, republished for the first time since 1820 that further tells of a darkness machine and a lunarian named Zuloc. Other sci-stories include the first r

# Pregunta 5

Iteramos para obtener las categorías de todas las películas

In [15]:
# Nro de estrellas
rating_classes = browser.find_elements(By.XPATH, '//ol/li/article/p')
def text_to_number(text):
    mapping = {
        "Zero": 0,
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5
    }
    return mapping.get(text, 0)

ratings = [text_to_number(rating_class.get_attribute('class').split()[1]) for rating_class in rating_classes]

# Disponibilidad
availabilities = [availability.text for availability in browser.find_elements(By.XPATH, '//ol/li/article/div[@class="product_price"]/p[2]')]

# Precios
prices = [price.text for price in browser.find_elements(By.XPATH, '//ol/li/article/div[@class="product_price"]/p[1]')]
prices = [re.search(r'(\d+\.\d+)', price).group(1) for price in prices]
# Títulos
titles = [title.get_attribute('title') for title in browser.find_elements(By.XPATH, '//ol/li/article/h3/a')]


# Itera sobre los elementos y obtiene el atributo 'href' de cada uno
links = [link.get_attribute('href') for link in browser.find_elements(By.XPATH, '//ol/li/article/h3/a')]


#descripciones
descriptions_list = []

for link in links:
    try:
        rl = requests.get(link)
        tree_detail = html.fromstring(rl.content)
        description_elements = tree_detail.xpath('//article/p')
        if description_elements:
            descriptions_list.append(description_elements[0].text)
        else:
            print(f"No se encontró descripción para el enlace: {link}")
    except Exception as e:
        print(f"Error procesando el enlace {link}: {e}")



# Crear DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Rating': ratings,
    'Availability': availabilities,
    'Price': prices,
    'Link': links,
    'Description': descriptions_list
})

df.head(16)

,Title,Rating,Availability,Price,Link,Description
0,Mesaerion: The Best Science Fiction Stories 18...,1,In stock,37.59,https://books.toscrape.com/catalogue/mesaerion...,"Andrew Barger, award-winning author and engine..."
1,Join,5,In stock,35.67,https://books.toscrape.com/catalogue/join_902/...,What if you could live multiple lives simultan...
2,"William Shakespeare's Star Wars: Verily, A New...",4,In stock,43.30,https://books.toscrape.com/catalogue/william-s...,MAY THE VERSE BE WITH YOU!Return once more to ...
3,The Project,1,In stock,10.65,https://books.toscrape.com/catalogue/the-proje...,"Welcome to Adrianna, a new country isolated fo..."
4,Soft Apocalypse,2,In stock,26.12,https://books.toscrape.com/catalogue/soft-apoc...,What happens when resources become scarce and ...
5,Sleeping Giants (Themis Files #1),1,In stock,48.74,https://books.toscrape.com/catalogue/sleeping-...,A page-turning debut in the tradition of Micha...
6,Arena,4,In stock,21.36,https://books.toscrape.com/catalogue/arena_587...,A fast-paced and gripping near-future science ...
7,Foundation (Foundation (Publication Order) #1),1,In stock,32.42,https://books.toscrape.com/catalogue/foundatio...,For twelve thousand years the Galactic Empire ...
8,The Restaurant at the End of the Universe (Hit...,1,In stock,10.92,https://books.toscrape.com/catalogue/the-resta...,Facing annihilation at the hands of the warlik...
9,Ready Player One,4,In stock,19.07,https://books.toscrape.com/catalogue/ready-pla...,"In the year 2044, reality is an ugly place. Th..."


# Pregunta 6

Si quiero convertir la data, siguiendo las pautas de la clase, en formato Json, debo convertir mi dataframe en listas de diccionarios

In [16]:
import json
list_data = df.to_dict(orient='records')
name='books_data'
path_fname = f'D:/DMC/PEA ADVANCED DATA SCIENCE/Web Scraping/tarea/output/{name}.json'
  # Reemplaza esto con la ruta donde deseas guardar el archivo
f = open(path_fname, 'w')
f.write(json.dumps(list_data, indent=4)) 
f.close()

![img30](https://github.com/rodrigosilupu/DMC_WS/blob/main/json_data.png?raw=true)

O, desde pandas directamente puedo guardar también a CSV

In [19]:
path_fname = f'D:/DMC/PEA ADVANCED DATA SCIENCE/Web Scraping/tarea/output/{name}.csv'
df.to_csv(path_fname, sep=";", index=False, encoding='utf-8')

![img40](https://github.com/rodrigosilupu/DMC_WS/blob/main/csv_data.png?raw=true)

![img5](https://github.com/rodrigosilupu/DMC_WS/blob/main/p6.png?raw=true)

In [18]:
browser.quit()